# 파이썬 데이터 모델

파이썬에서 데이터 모델이란 일종의 **프레임워크**로 간주할 수 있다. 시퀀스, 반복자, 함수, 코루틴 등 언어 자체를 구성하는 단위 간 인터페이스를 공식적으로 정의한다. 
파이썬에는 특별한 기능을 하는 메소드를 정의할 수 있다. 바로 `__*__` 를 이용하면 된다. 공식적으로는 **[던더]**(https://docs.python.org/3/reference/lexical_analysis.html#reserved-classes-of-identifiers)(System-defined names, informally known as “dunder” names) 라고 불리는 방식이다.

인터프리터는 이러한 특별 메소드를 호출해서 정해진 일을 수행한다. 밑의 예시와 함께 조금 더 자세하게 알아보자


In [7]:
import collections

# Card(rank, suit) => 일련의 속성으로만(attribute) 구성된 객체의 클래스를 정의할때 이용
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

    def __len__(self):
        """
        길이를 알려주는 특별 메소드
        """
        return len(self._cards)
    
    def __getitem__(self, position):
        """
        특정 포지션의 아이템을 가져오는 메소드
        """
        return self._cards[position]

In [8]:
Card(7, 'diamonds')

Card(rank=7, suit='diamonds')

In [4]:
deck = FrenchDeck()
deck

In [5]:
len(deck)

52

In [6]:
deck[0]

Card(rank='2', suit='spades')

In [9]:
from random import choice

"""
__getitem__ 과 함께 운영됨 => 정확히는 우리 메서드에서 __getitem__ 은 [] 에 작업을 위임함
"""
choice(deck)

Card(rank='7', suit='clubs')

In [10]:
deck[:3]

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

현재 `__getitem__()` 특별 메소드를 구현했으므로 반복문도 가능하다. 중요한 점은, 다른 객체 지향 언어처럼 상속을 이용하기 보다는 데이터 모델과 구성을 활용해서 구현했다는 점이다.
`__len__()` 과 `__getitem__()` 을 구현하기만 해도 파이썬 시퀀스 처럼 잘 작동할 수 있게 된다. 

In [12]:
for card in deck[:3]:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')


## __getiter__() 와 __getitem()__ 우선 순위

간단한 예시인데 `for-loop` 를 수행할때 과연 어떤 특별 메소드가 더 우선순위를 가지는 지 한번 코드 구현을 통해서 알아보도록 하자.

In [28]:
import collections

# Card(rank, suit) => 일련의 속성으로만(attribute) 구성된 객체의 클래스를 정의할때 이용
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck1:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self, length=10) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks][:length]
        self._index = 0
        self._length = length

    def __len__(self):
        """
        길이를 알려주는 특별 메소드
        """
        return len(self._cards)
    
    def __getitem__(self, position):
        """
        특정 포지션의 아이템을 가져오는 메소드
        """
        print('called __getitem__')
        return self._cards[position]
    
    def __iter__(self):
        print('called __iter__')
        while self._index < len(self._cards):
            print('yielding card {}'.format(self._index))
            card = self._cards[self._index]
            self._index += 1
            yield card


In [29]:
deck = FrenchDeck1()
for card in deck:
    card

called __iter__
yielding card 0
yielding card 1
yielding card 2
yielding card 3
yielding card 4
yielding card 5
yielding card 6
yielding card 7
yielding card 8
yielding card 9


결과를 보면 `__iter__()` 는 한번만 호출되는데 그 이유는 다음과 같다. 함수를 보면 `yield` 를 포함하고 있는데 이는 **제네레이터(generator)** 함수인데, 제네레이터 함수는 여러번 호출되는 것이 아니라 한번 호출될때 **제네레이터 객체(generator object)** 를 리턴한다.

그 이후로는 for-loop 내부에서 제네레이터 객체의 `__next__()` 메소드를 호출하여 반복문을 진행한다.  

```
called __iter__
yielding card 0
yielding card 1
yielding card 2
yielding card 3
yielding card 4
yielding card 5
yielding card 6
yielding card 7
yielding card 8
yielding card 9
```

다음에는 **수치형 모델**을 흉내내기 위해 여러 특별 메소드들을 구현해보자. 
`__repr__()`, `__abs()__`, `__add__()`, `__mul__()` 을 구현해 볼 것이다. 들어가기에 앞서 일단 `doctest` 를 사용해서 우리가 구현해야 할 부분에 대해 테스트를 주석과 함께 작성할 것이다. 아래 코드와 같이 `doctest()` 로 작성해두면 test 도 별도로 가능하다. 다만 현업에서는 주로 테스트 코드로서 대체하겠지만, 간단하게 예상되는 결과값을 표현할 수 있는 정도의 주석 프로토콜로 알아둬도 좋을 것 같다.

In [35]:
import doctest

def add(x, y):
    """
    >>> 1 + 2
    3
    """
    return x + y

doctest.testmod()

TestResults(failed=0, attempted=17)

In [36]:
import math

class Vector:
    """
    >>> v1 = Vector(1, 2)
    >>> v2 = Vector(3, 4)
    >>> v1 + v2
    Vector(4, 6)

    >>> v = Vector(3, 4)
    >>> abs(v)
    5.0

    >>> v * 3
    Vector(9, 12)

    >>> abs(v * 3)
    15.0
    """

    def __init__(self, x: int = 0, y: int = 0) -> None:
        self.x = x
        self.y = y

    def __repr__(self):
        """
        >>> v = Vector(3, 4)
        >>> v
        Vector(3, 4)
        """
        return f'Vector({self.x}, {self.y})'
    
    def __abs__(self):
        """
        >>> v = Vector(3, 4)
        >>> abs(v)
        5.0
        """
        return math.hypot(self.x, self.y)
    
    def __mul__(self, scalar):
        """
        >>> v = Vector(3, 4)
        >>> v * 3
        Vector(9, 12)
        """
        return Vector(self.x * scalar, self.y * scalar)
    
    def __add__(self, other: "Vector"):
        """
        >>> v1 = Vector(1, 2)
        >>> v2 = Vector(3, 4)
        >>> v1 + v2
        Vector(4, 6)
        """
        return Vector(self.x + other.x, self.y + other.y)

doctest.testmod()

TestResults(failed=0, attempted=17)

### 중위 연산자(Infix Operator)

위의 예시를 보면 **덧셈(+) 와 곱셈(*)**과 같은 중위연산자들이 존재한다. 여기서 중위 연산자의 특성을 알 수 있는데 기존 객체에 직접 수정(in-place)을 가하기 보다, 새로운 객체를 생성해낸다. 즉, 기존 객체를 바꾸지 않고, 새로운 객체를 뱉어 내는 것이다. 어렵게 적은 것 같지만 생각해보면 상식적인 일이다. 